## First attempt to play with xopt


## Load in a BELLA specific config file
read in a yaml file which has the control variable information and the objective variable information

In [25]:
# Ignore all warnings
import warnings
warnings.filterwarnings("ignore")

import torch
from xopt.vocs import VOCS
from xopt.evaluator import Evaluator
from xopt.generators.bayesian import ExpectedImprovementGenerator
from xopt import Xopt

import yaml
import pandas as pd
import numpy as np


#define a "yaml file" with no entries for variables. These will be filled in used the "bella_config" yamml file
YAML = """
xopt:
    dump_file: dump.yaml
generator:
    name: upper_confidence_bound
    n_initial: 5
    optim:
        num_restarts: 1
    acq:
        proximal_lengthscales: []

evaluator:
    function: __main__.geecs_measurement

vocs:
    variables:
        {}
    objectives: {f: "MAXIMIZE"}

"""
yaml_output = yaml.safe_load(YAML)

with open("HTU_config.yaml", "r") as stream:
    try:
        bella_config=yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)
        
#parse the config file to create a dict with the variables in proper format for xopt
cv_tags=list(bella_config['control_variables'].keys())
#obj_tags=list(bella_config['objective_variables'].keys())

ranges=[]
obj_tags=[]
# for i in cv_tags:
#     if bella_config['control_variables'][i]['set']:
#         ranges.append(bella_config['control_variables'][i]['range'])
#         #get the normalization constants for the variable range. Normalizing the range seems to speed things up a bit
#         range_offset=[0,bella_config['control_variables'][i]['range'][1]-bella_config['control_variables'][i]['range'][0]]
#         range_norm=[0,1]
#         print(range_offset)
# #         yaml_output['vocs']['variables'][i]=bella_config['control_variables'][i]['range']
#         yaml_output['vocs']['variables'][i]=[0,1]

#         yaml_output['generator']['acq']['proximal_lengthscales']+=[bella_config['control_variables'][i]['proximal_lengthscale']/range_offset[1]]
#     if bella_config['control_variables'][i]['objective']:
#         obj_tags.append(i)
    

cv_tags=dict(bella_config['control_variables'])
for cv_tag, cv_config in cv_tags.items():
    if cv_config['set']:
        ranges.append(cv_config['range'])
        range_offset=[0,cv_config['range'][1]-cv_config['range'][0]]
        range_norm=[0,1]
        print(range_offset)
        #yaml_output['vocs']['variables'][cv_tag]=cv_config['range']
        yaml_output['vocs']['variables'][cv_tag]=[0,1]
        yaml_output['generator']['acq']['proximal_lengthscales'].append(cv_config['proximal_lengthscale']/range_offset[1])
    if cv_config['objective']:
        obj_tags.append(cv_tag)
        
        
# for i in obj_tags:
#     yaml_output['vocs']['objectives'][i]=bella_config['objective_variables'][i]['method']
    
print(yaml_output)

#until I figure out the above, we will need to initialize the requisite Xopt objects
#individually

# vocs = VOCS(
#     variables=yaml_output['vocs']['variables'],
#     objectives=yaml_output['vocs']['objectives'],
# )

# print(vocs)
print(obj_tags)

[0, 5.0]
[0, 3.0]
[0, 15.0]
[0, 2000.0]
{'xopt': {'dump_file': 'dump.yaml'}, 'generator': {'name': 'upper_confidence_bound', 'n_initial': 5, 'optim': {'num_restarts': 1}, 'acq': {'proximal_lengthscales': [0.3, 0.5, 0.1, 0.25]}}, 'evaluator': {'function': '__main__.geecs_measurement'}, 'vocs': {'variables': {'JetX': [0, 1], 'JetY': [0, 1], 'JetZ': [0, 1], 'GratingSeparation': [0, 1]}, 'objectives': {'f': 'MAXIMIZE'}}}
['Charge']


## define the get/set functions for BELLA GEECS 


In [26]:
def client_factory(ip, port, var):
    print('in the client factory for device: ',var)
    client=socket.socket(socket.AF_INET, socket.SOCK_STREAM);
    client.connect((str(ip),int(port)))
    #client.setblocking(0)
    subcriptionstring = bytes('Wait>>'+str(var),'ascii')
    #get length of subscription message
    SubcriptionCmdLength = len(subcriptionstring)
    #Flatten the length of the subscription message length
    sizepack = struct.pack('>i', SubcriptionCmdLength)
    #Send the size of the message followed by the message
    client.sendall( sizepack + subcriptionstring)
    return client   

def get(tag):
    #can change how get is defined but use this function elsewhere
    value=get_tcp_nonblocking(tag)
    return value

def get_until_new(tag):
    attempts=0
    while attempts<6:
        val=get(tag)
        if bella_config['control_variables'][i]['newDataFlag']==1:
            break
        else:
            time.sleep(0.2)
            attempts=attempts+1
    if attempts>5:
        val=0.0
    return val

def get_tcp_nonblocking(tag):    
    #info('function get1')
    
    #start by trying to check out a socket so that when a process calls  
    #to get the value, you don't have multiple attempts to read/clear the buffer.
    #If the socket isn't currently busy, swith it to "busy" until finished
    if bella_config['control_variables'][tag]['busy']==0: 
        #print("socket was clear when requested")
        bella_config['control_variables'][tag]['busy']=1
        if False: #skipping
            #print('objective function')
            f(x)
            if hasattr(y, '__iter__'):
                return y[0]
            else:
                return y
        else:
            client=bella_config['control_variables'][tag]['tcp_client']
            #print("got client: ",client)
            dt=0
            counter=0
            #note: the dt defined below should be shorter than the timeout in the select.select command
            #The select.select command asks the client if there is any information to transmist. If there
            #is, it returns true. If there is not any information after the timeout, it reports false.
            #Typical response time when a device has information to transmit is well below 1 ms. So, we rely
            # on the timeout to tell us that there is no information on the buffer, and we are waiting on 
            # another iteration of the device's acquire loop.
            while dt<0.0015:
                counter=counter+1
                t0=time.monotonic()
                ready=select.select([client],[],[],.002 ) #last arguement is timeout in seconds
                #print(ready)
                if ready[0]:
                    size = struct.unpack('>i', client.recv(4))[0]  # Extract the msg size from four bytes - mind the encoding
                    str_data = client.recv(size)
                    geecs=str_data.decode('ascii').split(",")
                    #print(geecs)
                    geecs=geecs[-2].split(" ")[0]
                    #print(geecs)
                    if len(geecs)==0:
                        geecs="nan"
                    if geecs=='on':
                        geecs=1
                    if geecs=='off':
                        geecs=0
                    #print(geecs)
                    if type(geecs) ==  str:
                        if any(c.isalpha() for c in geecs):
                            geecs=0
                    bella_config['control_variables'][tag]['actual_value']=geecs
                    #print(bella_config['control_variables'][tag]['value'])
                    bella_config['control_variables'][tag]['newDataFlag']=1
                    #print("chewing through TCP buffer. Device value: ",geecs)
                else:
                    #print("Buffer cleared")
                    if counter==1:
                        geecs=bella_config['control_variables'][tag]['actual_value']
                        bella_config['control_variables'][tag]['newDataFlag']=0
                t1=time.monotonic()
                dt=t1-t0
            bella_config['control_variables'][tag]['busy']=0 #release the socket
            #print("socket released")
    else:
        print("socket was busy when requested")
        geecs=bella_config['control_variables'][tag]['actual_value']
        bella_config['control_variables'][tag]['newDataFlag']=0
        bella_config['control_variables'][tag]['busy']=0
        print("new data: ",newDataFlags[index])
    #print("in get1 gotvalue ans index "+str(gotValues[index])+' '+str(index))
    return geecs

def set1(tag, value):
    #check that the set value is in the range 
    if value>bella_config['control_variables'][tag]['range'][0] and value<bella_config['control_variables'][tag]['range'][1]:
        
        attempts=0
        while attempts<3:
            print('attempts loop')
            UDP_IP = str(bella_config['control_variables'][tag]['ipaddress'])
            UDP_PORT = int(bella_config['control_variables'][tag]['port'])
            position = round(value,4)
            #MESSAGE = bytes("set"+str(bella_config['control_variables'][tag]['device_variable'])+">>" + str(position), 'ascii')
            MESSAGE = f"set{bella_config['control_variables'][tag]['device_variable']}>>{value:.4f}".encode('ascii')
            sock = socket.socket(socket.AF_INET, # Internet
                    socket.SOCK_DGRAM) # UDP
            sock.sendto(MESSAGE, (UDP_IP, UDP_PORT))
            time.sleep(0.75)
            checkVal=get(tag)
            print("check value is: "+str(checkVal)+" and target value is: "+str(value))
            tolerance=abs((float(checkVal)-float(value))/float(value))
            timeout=5
            newCheckVal='NaN';
            #loop to check value while device is moving. if value doesn't reach desired value, re-attempt to send the command
            while tolerance>0.01:
                print('tolerance loop')
                time.sleep(0.75)
                checkVal=get(tag)
                tolerance=abs((float(checkVal)-float(value))/float(value))
                if newCheckVal==checkVal:
                    print('failed to move device',tag)
                    attempts+=1
                    break
                newCheckVal=checkVal
                print("still moving, check value is: "+str(checkVal)+" and target value is: "+str(value))
            if tolerance<0.01:
                print('successfully moved device',tag)
                attempts=4
    else:
        print("set value of "+str(value)+"for "+str(tag)+" is not in the range: "+str(bella_config['control_variables'][tag]['range']))




def set1Norm(tag, norm_value):
    #check that the set value is in the range 
    value=norm_value*(bella_config['control_variables'][tag]['range'][1]-bella_config['control_variables'][tag]['range'][0])+bella_config['control_variables'][tag]['range'][0]
    print(value)
#     if value>bella_config['control_variables'][tag]['range'][0] and value<bella_config['control_variables'][tag]['range'][1]:
        
#         attempts=0
#         while attempts<3:
#             print('attempts loop')
#             UDP_IP = str(bella_config['control_variables'][tag]['ipaddress'])
#             UDP_PORT = int(bella_config['control_variables'][tag]['port'])
#             position = round(value,4)
#             #MESSAGE = bytes("set"+str(bella_config['control_variables'][tag]['device_variable'])+">>" + str(position), 'ascii')
#             MESSAGE = f"set{bella_config['control_variables'][tag]['device_variable']}>>{value:.4f}".encode('ascii')
#             sock = socket.socket(socket.AF_INET, # Internet
#                     socket.SOCK_DGRAM) # UDP
#             sock.sendto(MESSAGE, (UDP_IP, UDP_PORT))
#             time.sleep(0.75)
#             checkVal=get(tag)
#             print("check value is: "+str(checkVal)+" and target value is: "+str(value))
#             tolerance=abs((float(checkVal)-float(value))/float(value))
#             timeout=5
#             newCheckVal='NaN';
#             #loop to check value while device is moving. if value doesn't reach desired value, re-attempt to send the command
#             while tolerance>0.01:
#                 print('tolerance loop')
#                 time.sleep(0.75)
#                 checkVal=get(tag)
#                 tolerance=abs((float(checkVal)-float(value))/float(value))
#                 if newCheckVal==checkVal:
#                     print('failed to move device',tag)
#                     attempts+=1
#                     break
#                 newCheckVal=checkVal
#                 print("still moving, check value is: "+str(checkVal)+" and target value is: "+str(value))
#             if tolerance<0.01:
#                 print('successfully moved device',tag)
#                 attempts=4
#     else:
#         print("set value of "+str(value)+"for "+str(tag)+" is not in the range: "+str(bella_config['control_variables'][tag]['range']))




In [27]:
for cv_tag,cv_config in cv_tags.items():
    print(cv_config)

{'device_name': 'U_ESP_JetXYZ', 'device_variable': 'Position.Axis 1', 'range': [2.0, 7.0], 'ipaddress': '0.0.0.0', 'port': 9999, 'busy': 0, 'newDataFlag': 0, 'tcp_client': 'na', 'actual_value': 0.0, 'set': True, 'set_value': 0.0, 'proximal_lengthscale': 1.5, 'objective': False}
{'device_name': 'U_ESP_JetXYZ', 'device_variable': 'Position.Axis 2', 'range': [-8.0, -5.0], 'ipaddress': '0.0.0.0', 'port': 9999, 'busy': 0, 'newDataFlag': 0, 'tcp_client': 'na', 'actual_value': 0.0, 'set': True, 'set_value': 0.0, 'proximal_lengthscale': 1.5, 'objective': False}
{'device_name': 'U_ESP_JetXYZ', 'device_variable': 'Position.Axis 3', 'range': [10.0, 25.0], 'ipaddress': '0.0.0.0', 'port': 9999, 'busy': 0, 'newDataFlag': 0, 'tcp_client': 'na', 'actual_value': 0.0, 'set': True, 'set_value': 0.0, 'proximal_lengthscale': 1.5, 'objective': False}
{'device_name': 'U_CompAerotech', 'device_variable': 'Position.Axis1', 'range': [40000.0, 42000.0], 'ipaddress': '0.0.0.0', 'port': 9999, 'busy': 0, 'newDataFl

## get TCP info from database for variables, establish clients, initialize current values
skip this part if running in simulation mode

In [28]:
#get TCP info from database
import mysql.connector
import socket
import struct
import time
import select

mydb = mysql.connector.connect(
host=bella_config['database']['ipaddress'],
user=bella_config['database']['user'],
password=bella_config['database']['password'])

selectors=["ipaddress","commport"]
selectorString=",".join(selectors)

mycursor = mydb.cursor()
db_name=bella_config['database']['name']

# for i in cv_tags:
#     #create a sql query command that looks in the 'device' table to find the IP address and port for a given device name
#     mycursor.execute("SELECT "+selectorString+" FROM "+db_name+".device where name="+'"' + bella_config['control_variables'][i]['device_name'] + '"'+";")
#     myresult = list(mycursor.fetchall()[0])
#     print(myresult)
#     bella_config['control_variables'][i]['ipaddress']=myresult[0]
#     bella_config['control_variables'][i]['port']=myresult[1]
#     bella_config['control_variables'][i]['tcp_client']=client_factory(myresult[0],myresult[1],bella_config['control_variables'][i]['device_variable'])
#     time.sleep(1)
#     get(i)
    
for cv_tag,cv_config in cv_tags.items():
    #create a sql query command that looks in the 'device' table to find the IP address and port for a given device name
    select_stmt="SELECT "+selectorString+" FROM "+db_name+".device where name="+'"' + cv_config['device_name'] + '"'+";"

    mycursor.execute(select_stmt)
    myresult = list(mycursor.fetchall()[0])
    print(myresult)
    cv_config['ipaddress']=myresult[0]
    cv_config['port']=myresult[1]
    cv_config['tcp_client']=client_factory(myresult[0],myresult[1],cv_config['device_variable'])
    time.sleep(1)
    get(cv_tag)
    
# for i in obj_tags:
#     #create a sql query command that looks in the 'device' table to find the IP address and port for a given device name
#     mycursor.execute("SELECT "+selectorString+" FROM "+db_name+".device where name="+'"' + bella_config['objective_variables'][i]['device_name'] + '"'+";")
#     myresult = list(mycursor.fetchall()[0])
#     print(myresult)
#     bella_config['objective_variables'][i]['ipaddress']=myresult[0]
#     bella_config['objective_variables'][i]['port']=myresult[1]
#     bella_config['objective_variables'][i]['tcp_client']=client_factory(myresult[0],myresult[1],bella_config['objective_variables'][i]['device_variable'])
#     get(i)
    

    
for cv_tag,cv_config in cv_tags.items():
    print(cv_config['actual_value'])
    
# for i in obj_tags:
#     print(bella_config['objective_variables'][i]['actual_value'])

# for i in cv_tags:
#     bella_config['control_variables'][i]['tcp_client'].close()

['192.168.8.198', '65158']
in the client factory for device:  Position.Axis 1
['192.168.8.198', '65158']
in the client factory for device:  Position.Axis 2
['192.168.8.198', '65158']
in the client factory for device:  Position.Axis 3
['192.168.6.73', '65119']
in the client factory for device:  Position.Axis1
['192.168.6.100', '64855']
in the client factory for device:  ai_mean.channel 0
4.500270
-6.000010
14.999570
43099.368750
3.565506793908


## define a simulated objective function


In [83]:
def gaussian(x, mu, sig):
    return np.exp(-np.power(x - mu, 2.) / (2 * np.power(sig, 2.)))

#simulate objective using actual values
# def simulate_objective1():
#     v1=gaussian(float(bella_config['control_variables']['JetX']['actual_value']),5.5,.5)+np.random.normal(0,.05,1)[0]
#     v2=gaussian(float(bella_config['control_variables']['JetY']['actual_value']),-7.0,.5)+np.random.normal(0,.05,1)[0]
#     v3=gaussian(float(bella_config['control_variables']['JetZ']['actual_value']),15.0,1)+np.random.normal(0,.05,1)[0]
#     v4=gaussian(float(bella_config['control_variables']['GratingSeparation']['actual_value']),43.1000,0.0500)+np.random.normal(0,.05,1)[0]
#     res=v1*v2*v4*v4
#     res=float("%.4f" % res)
#     return res


#simulate objective using set values. 
def simulate_objective2_norm(input_dict):
    v1=gaussian(float(input_dict['JetX']),0.5,.3)+np.random.normal(0,.05,1)[0]
    v2=gaussian(float(input_dict['JetY']),0.2,.5)+np.random.normal(0,.05,1)[0]
    v3=gaussian(float(input_dict['JetZ']),0.4,.1)+np.random.normal(0,.05,1)[0]
    v4=gaussian(float(input_dict['GratingSeparation']),0.9,.25)+np.random.normal(0,.05,1)[0]
    
    v5=0.8*gaussian(float(input_dict['JetX']),0.5+.2,.3)+np.random.normal(0,.05,1)[0]
    v6=0.8*gaussian(float(input_dict['JetY']),0.2+.2,.5)+np.random.normal(0,.05,1)[0]
    v7=0.8*gaussian(float(input_dict['JetZ']),0.4+.2,.1)+np.random.normal(0,.05,1)[0]
    v8=0.8*gaussian(float(input_dict['GratingSeparation']),0.9-.2,.25)+np.random.normal(0,.05,1)[0]
    res=v1*v2*v3*v4+v5*v6*v7*v8
    res=float("%.4f" % res)
    return res


    
        

In [84]:
outputs=obj_tags

def geecs_measurement(input_dict):

    for i in list(input_dict.keys()):
        set_val=float(input_dict[i])
        #print('set '+str(i)+' to '+str(set_val))
        #set1(i,float(set_val))
    
    outs=[]
    counter=0
# use while loop below for simulating the objective function
    while counter<bella_config['optimization_parameters']['shots_per_step']:
        for i in outputs:
            val=simulate_objective2_norm(input_dict)
            outs.append(val)
        counter=counter+1

#Use below for making actual maeasurements on beamline
#     while counter<bella_config['optimization_parameters']['shots_per_step']:
#         for i in outputs:
#             val=float(get_until_new(i))
#             outs.append(val)
#         counter=counter+1
        
#     print(outs)

###############
## TO DO
###############
## Add some data processing on the resulting on the acquired data.
## for example, remove the extreme values, then maybe take a median
## also need to test with passing all acquired data points rather than some stat

    return {'f': np.mean(outs)}


In [86]:
#close all tcp clients
# for i in cv_tags:
#     bella_config['control_variables'][i]['tcp_client'].close()

## Create Xopt objects
Create the evaluator to evaluate our test function and create a generator that uses
the Expected Improvement acqusition function to perform Bayesian Optimization.

In [85]:
from xopt.evaluator import Evaluator
from xopt.generators.bayesian import ExpectedImprovementGenerator
from xopt import Xopt

X = Xopt(config=yaml_output)


# evaluator = Evaluator(function=geecs_measurement)
# generator = ExpectedImprovementGenerator(vocs)

# X = Xopt(evaluator=evaluator, generator=generator, vocs=vocs)
# X.options.dump_file='dump.yaml'
X



            Xopt
________________________________
Version: 1.2.1
Data size: 0
Config as YAML:
xopt: {asynch: false, strict: false, dump_file: dump.yaml, max_evaluations: null}
generator:
  name: upper_confidence_bound
  optim: {num_restarts: 1, raw_samples: 20, sequential: true, use_nearby_initial_points: true,
    max_travel_distances: null}
  acq:
    proximal_lengthscales: [0.3, 0.5, 0.1, 0.25]
    use_transformed_proximal_weights: true
    monte_carlo_samples: 128
    beta: 2.0
  model:
    function: xopt.generators.bayesian.models.standard.create_standard_model
    kwargs: {use_conservative_prior_lengthscale: false, use_conservative_prior_mean: false,
      use_low_noise_prior: false}
  n_initial: 5
evaluator:
  function: __main__.geecs_measurement
  max_workers: 1
  function_kwargs: {}
  vectorized: false
vocs:
  variables:
    JetX: [0.0, 1.0]
    JetY: [0.0, 1.0]
    JetZ: [0.0, 1.0]
    GratingSeparation: [0.0, 1.0]
  constraints: {}
  objectives: {f: MAXIMIZE}
  constants: {

## Generate and evaluate initial points
To begin optimization, we must generate some random initial data points. The first call
to `X.step()` will generate and evaluate a number of randomly points specified by the
 generator. Note that if we add data to xopt before calling `X.step()` by assigning
 the data to `X.data`, calls to `X.step()` will ignore the random generation and
 proceed to generating points via Bayesian optimization.

In [82]:
X.evaluate_data(pd.DataFrame({"JetX":[.1, .7],"JetY":[.1, .4],"JetZ":[.1, .55],'GratingSeparation':[.1, .7]}))
#X.evaluate_data(pd.DataFrame({"JetX":[4.1, 5.4],"JetY":[-6.1, -7.1],"JetZ":[10.1, 14.35],'GratingSeparation':[40500.1, 41100.8]}))

,JetX,JetY,JetZ,GratingSeparation,f,xopt_runtime,xopt_error
1,0.1,0.1,0.10,0.1,-0.000320,0.005541,False
2,0.7,0.4,0.55,0.7,0.526995,0.004295,False


In [72]:
# # print initial number of points to be generated
# print(X.generator.options.n_initial)

# # call X.step() to generate + evaluate initial points
# X.step()

# # inspect the gathered data
# X.data

## Do bayesian optimization steps
To perform optimization we simply call `X.step()` in a loop. This allows us to do
intermediate tasks in between optimization steps, such as examining the model and
acquisition function at each step (as we demonstrate here).

In [77]:
import torch
import matplotlib.pyplot as plt
n_steps = 100

# test points for plotting
#test_x = torch.linspace(*X.vocs.bounds.flatten(),50).double()

for i in range(n_steps):
    print('step'+str(i))
    t0=time.monotonic()
    X.step()
    t1=time.monotonic()
    print('step'+str(i)+".1: "+str(t1-t0))
    # get the Gaussian process model from the generator
    model = X.generator.model
    t2=time.monotonic()
    print('step'+str(i)+".2: "+str(t2-t0))
    # get acquisition function from generator
    acq = X.generator.get_acquisition(model)
    t3=time.monotonic()
    print('step'+str(i)+".3: "+str(t3-t0))
    # calculate model posterior and acquisition function at each test point
    # NOTE: need to add a dimension to the input tensor for evaluating the
    # posterior and another for the acquisition function, see
    # https://botorch.org/docs/batching for details
    # NOTE: we use the `torch.no_grad()` environment to speed up computation by
    # skipping calculations for backpropagation
#     with torch.no_grad():
#         posterior = model.posterior(test_x.unsqueeze(1))
#         acq_val = acq(test_x.reshape(-1,1,1))
#     # get mean function and confidence regions
#     mean = posterior.mean
#     l,u = posterior.mvn.confidence_region()
#     # plot model and acquisition function
#     fig,ax = plt.subplots(2,1,sharex="all")
#     # plot model posterior
#     ax[0].plot(test_x, mean, label="Posterior mean")
#     ax[0].fill_between(test_x, l, u,alpha=0.25, label="Posterior confidence region")
#     # add data to model plot
#     ax[0].plot(X.data["x"],X.data["f"],"C1o", label="Training data")
#     # plot true function
#     true_f = sin_function({"x": test_x})["f"]
#     ax[0].plot(test_x, true_f,'--', label="Ground truth")
#     # add legend
#     ax[0].legend()
#     # plot acquisition function
#     ax[1].plot(test_x, acq_val.flatten())
#     ax[0].set_ylabel("f")
#     ax[1].set_ylabel(r"$\alpha(x)$")
#     ax[1].set_xlabel("x")


step0
step0.1: 1.4997008339996682
step0.2: 1.4997828339996886
step0.3: 1.5003434999998717
step1
step1.1: 2.2817978329999278
step1.2: 2.2818839579999803
step1.3: 2.2824540419996993
step2
step2.1: 1.5552208340000107
step2.2: 1.5553673340000387
step2.3: 1.5561113339999793
step3
step3.1: 1.6903213750001669
step3.2: 1.6904070410000713
step3.3: 1.690944416000093
step4
step4.1: 1.550165458000265
step4.2: 1.5502518329999475
step4.3: 1.5507722080001258
step5
step5.1: 1.5021570830003839
step5.2: 1.502244083000278
step5.3: 1.5027890000001207
step6
step6.1: 1.4740004170002976
step6.2: 1.4740867920004348
step6.3: 1.4746346670003732
step7
step7.1: 1.5691754169997694
step7.2: 1.569255666999652
step7.3: 1.5698924999996962
step8
step8.1: 1.9468914580002092
step8.2: 1.9469727910000074
step8.3: 1.9475756660003753
step9
step9.1: 2.3279111249999005
step9.2: 2.3280247080001573
step9.3: 2.328643207999903
step10
step10.1: 1.829482999999982
step10.2: 1.8295660420003514
step10.3: 1.8302056249999623
step11
step1

In [68]:
# access the collected data
X.data


,JetX,JetY,JetZ,GratingSeparation,f,xopt_runtime,xopt_error
1,0.100000,0.100000,0.100000,0.100000,0.000040,0.007169,False
2,0.700000,0.400000,0.550000,0.700000,0.369750,0.004751,False
3,0.916034,0.400043,0.550004,0.701160,0.274930,0.001138,False
4,0.772193,0.400146,0.550012,0.821049,0.008485,0.001132,False
5,0.650277,0.400241,0.550021,0.643077,0.015795,0.001104,False
...,...,...,...,...,...,...,...
98,0.686048,0.292171,0.550167,0.409514,-0.004735,0.001134,False
99,0.688642,0.297042,0.550152,0.694989,0.365925,0.001095,False
100,0.692023,0.301953,0.550138,0.696761,0.368005,0.001091,False
101,0.692495,0.306670,0.550124,0.696369,0.356335,0.001098,False


## Getting the optimization result
To get the ideal point (without evaluating the point) we ask the generator to
generate a new point.

In [40]:
X.generator.generate(1)

,GratingSeparation,JetX,JetY,JetZ
0,0.820811,0.492083,0.644918,0.800003


In [42]:
X.evaluate_data(pd.DataFrame({"JetX":[.5],"JetY":[.64],"JetZ":[.8],'GratingSeparation':[.82]}))

,JetX,JetY,JetZ,GratingSeparation,f,xopt_runtime,xopt_error
44,0.5,0.64,0.8,0.82,-0.000155,0.00456,False


## Customizing optimization
Each generator has a set of options that can be modified to effect optimization behavior

In [ ]:
X.generator.options.dict()

In [ ]:
# example: add a Gamma(1.0,10.0) prior to the noise hyperparameter to reduce model noise
# (good for optimizing noise-free simulations)
X.generator.options.model.kwargs.use_low_noise_prior = True